In [ ]:
%pip install dash
%pip install plotly.express
%pip install pandas

In [2]:
import pandas as pd
from datetime import date
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
#from dash.dependencies import Input, Output

### plot

In [3]:
topic_df = pd.read_csv('./TopicsofReviews.csv')
date_df = pd.read_csv('./combined_data.csv')

# merge dfs into data
data = date_df.merge(topic_df, on='review', how='inner')

# clean date
data['date_clean'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
data['date_clean'] = data['date_clean'].combine_first(pd.to_datetime(data['date'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce'))

# sanity check that all dates are cleaned
# sum(data['date_clean'].isnull())# should output 0

data['date_clean'] = data['date_clean'].astype(str)
data['date_clean'] = data['date_clean'].str[:10]
data['date_clean'] = pd.to_datetime(data['date_clean']).dt.date    

# select columns needed
df = data[['review', 'date_clean', 'Topic_Name', 'Topic_Number']]

In [ ]:
get_topic_name = {
    0:'App Responsiveness', 1:'Money Growth (Interest Rates)', 2:'Customer Services', 
    3:'Services & Products', 4:'User Interface', 5:'Credit card usage', 
    6:'Login & Account Setup', 7:'Competition', 8:'Safety', 9:'Customer trust'
    }

# split reviews with multiple topics into duplicates of single topics
df2 = df[df['Topic_Number'].str.contains(',', regex=False)]


# hard coding cos splitting by regex is killing me
df['Topic_Number'] = df['Topic_Number'].str[0]
df['Topic_Number'] = df['Topic_Number'].astype(int)
df['Topic_Name'] = df['Topic_Number'].map(get_topic_name)

df2['Topic_Number'] = df2['Topic_Number'].str[2]
df2['Topic_Number'] = df2['Topic_Number'].astype(int)
df2['Topic_Name'] = df2['Topic_Number'].map(get_topic_name)

df = pd.concat([df,df2])

df = df.groupby(['Topic_Name', 'Topic_Number', 'date_clean']).size().reset_index(name='num_reviews')

In [18]:
min_date = df.date_clean.min()
max_date = df.date_clean.max()
print(min_date)
print(max_date)

s = df.date_clean[5]
e = df.date_clean[10]
print(s)
print(e)

2022-09-03
2024-03-10
2023-01-24
2023-07-19


In [19]:
topics_over_time = px.line(df, 'date_clean', 'num_reviews', color='Topic_Name', 
                           labels={"date_clean":"Date", "num_reviews":"Number of Reviews", "Topic_Name":"Topic"}, 
                           title="Number of Reviews by Topics over time")
# update date range
topics_over_time.update_xaxes(range=[s,e])

topics_over_time.show()

### dash

In [16]:
app = Dash(__name__)

app.layout = html.Div([
    dcc.DatePickerRange(
        id='tot_date_picker_range',
        min_date_allowed=min_date,
        max_date_allowed=max_date,
        initial_visible_month=min_date,
        end_date=max_date
    ),
    html.Div(id='output-container-date-picker-range'),
    dcc.Graph(id='tot', figure=topics_over_time)
])

@callback(
    Output('tot', 'figure'),
    Output('output-container-date-picker-range', 'children'),
    Input('tot_date_picker_range', 'start_date'),
    Input('tot_date_picker_range', 'end_date'))
    
def update_tot(start_date, end_date):
    if (start_date is not None) and (end_date is not None):
        tot = topics_over_time
        tot.update_xaxes(range=[start_date,end_date])
        return tot

def update_output_date_picker(start_date, end_date):
    string_prefix = 'You have selected: '
    if start_date is not None:
        start_date_object = date.fromisoformat(start_date)
        start_date_string = start_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
    if end_date is not None:
        end_date_object = date.fromisoformat(end_date)
        end_date_string = end_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        return string_prefix


if __name__ == '__main__':
    app.run(debug=True)


---------------------------------------------------------------------------
SchemaTypeValidationError                 Traceback (most recent call last)
SchemaTypeValidationError: Schema: [<Output `tot.figure`>, <Output `output-container-date-picker-range.children`>]
Path: ()
Expected type: (<class 'tuple'>, <class 'list'>)
Received value of type <class 'NoneType'>:
    None

---------------------------------------------------------------------------
SchemaTypeValidationError                 Traceback (most recent call last)
SchemaTypeValidationError: Schema: [<Output `tot.figure`>, <Output `output-container-date-picker-range.children`>]
                Path: ()
                Expected type: (<class 'tuple'>, <class 'list'>)
                Received value of type <class 'plotly.graph_objs._figure.Figure'>:
                    Figure({
    'data': [{'hovertemplate': 'Topic=App Responsiveness<br>Date=%{x}<br>Number of Reviews=%{y}<extra></extra>',
              'legendgroup': 'App Respon